In [1]:
#imports
from vpython import *
from math import *
import numpy as np
import random as r

<IPython.core.display.Javascript object>

In [2]:
#functions
#waits for user to push the key given
def waitforuser(key):
    while True:
        ev = scene.waitfor('keydown')
        
        if ev.key == 'i':
            break

#calculates distance between two 2d points
def distance(x1:float, y1:float, x2:float, y2:float):
    return sqrt((x2 - x1) ** 2 + (y1 - y2) ** 2)

#calculates cost a point obstacle creates
def obstacle_cost(ped_x:float, ped_y:float, ped_rad:float, ob_x:float, ob_y:float, ob_rad:float, rad:float):
    #gets distance from obstacle
    #subtract obstacle and pedestrian shape radi for more accurate distance
    dst = distance(ped_x, ped_y, ob_x, ob_y) - (ob_rad + ped_rad)
    
    #if distance is less than 0, make near 0
    if dst <= 0:
        dst = 0.00000001
    
    #if distance is less than radius, cost is ln(rad/dst)
    return (dst > 0 and dst <= rad) * log(rad/dst)

#calculates cost create by box
def box_cost(ped_x:float, ped_y:float, ped_rad:float, ob, ob_rad:float, rad:float):
    #declare theta angle between box axis and distance vector
    theta = 0
    
    #get distance vector, axis vector, and axis orthogonal vector
    dst_v = vector(ped_x - ob.pos.x, 0, ped_y - ob.pos.z)
    axis_v = vector(ob.axis.x, 0, ob.axis.z)
    axis_iv = vector(0,0,0)
    
    #if axis vector is (x,0,0) or (0,0,z), swap x and z
    if axis_v.x == 0 or axis_v.z == 0:
        axis_iv.x = axis_v.z
        axis_iv.z = axis_v.x
    #else find orthognal vector    
    else:
        axis_iv.x = 1 - 2 * (axis_v.x < 0)
        axis_iv.z = axis_v.x / axis_v.z * -1
        axis_iv = axis_iv.norm()
    
    #get product of axis and distance and of orthognal and distance
    mag_prod = axis_v.mag * dst_v.mag
    imag_prod = axis_iv.mag * dst_v.mag
    
    #get cos and sin values between axis/distance and orthogonal/distance
    cos_theta = acos(axis_v.dot(dst_v) / mag_prod)
    sin_theta = asin(axis_v.cross(dst_v).mag / mag_prod)
    icos_theta = acos(axis_iv.dot(dst_v) / imag_prod)
    isin_theta = asin(axis_iv.cross(dst_v).mag / imag_prod)
    
    #use relationship of both pairs of cos and sin to find accurate theta between axis and distance
    if ((cos_theta <= pi/2 and sin_theta >= 0) and (icos_theta <= pi/2 and isin_theta >= 0)):
        theta = 2*pi - sin_theta
    elif ((cos_theta >= pi/2 and sin_theta >= 0) and (icos_theta <= pi/2 and isin_theta >= 0)):
        theta = pi + sin_theta
    elif ((cos_theta >= pi/2 and sin_theta >= 0) and (icos_theta >= pi/2 and isin_theta >= 0)):
        theta = cos_theta
    elif ((cos_theta <= pi/2 and sin_theta >= 0) and (icos_theta >= pi/2 and isin_theta >= 0)):
        theta = cos_theta
    
    #get l and w of triangle that finds point nearest to ped
    l = dst * cos(theta)
    w = dst * sin(theta)
    
    #clamps values for l and w
    if l > ob.length/2:
        l = ob.length/2
    elif l < ob.length/2 * -1:
        l = ob.length/2 * -1
    if w > ob.width/2:
        w = ob.width/2
    elif w < ob.width/2 * -1:
        w = ob.width/2 * -1
     
    #get angle between axis of vector and x axis
    phi = vector(1,0,0).diff_angle(ob.axis)
    
    #rotate l and w so point in x and y
    x = ob.pos.x + cos(phi) * l - sin(phi) * w
    y = ob.pos.z + sin(phi) * l + cos(phi) * w
      
    #get cost from point nearest to ped
    return obstacle_cost(ped_x, ped_y, ped_rad, x, y, ob_rad, rad)

#gets overall cost at a location
def cost(ped_x:float, ped_y:float, target, ep_x:float, ep_y:float, peds:list, ped_cost:float, obs:list, ob_cost:float, bxs:list, bx_rad:float, bx_cost:float, d):
    #get cost relative to goal and declare f
    dst = distance(ped_x, ped_y, ep_x, ep_y)
    f = 0
    
    #get costs for all obstacles
    for ob in obs:
        f += obstacle_cost(ped_x, ped_y, target.radius, ob.pos.x, ob.pos.z, ob.radius, ob_cost)
        
    #get costs from social distance  
    for ped in peds:
        if ped == target:
            continue
        
        f += obstacle_cost(ped_x, ped_y, target.radius, ped.pos.x, ped.pos.z, ped.radius, ped_cost)
    
    #get costs from walls
    for bx in bxs:
        f += box_cost(ped_x, ped_y, target.radius, bx, bx_rad, bx_cost)
    
    #return total
    return dst + f

#estimates min cost path to goal
def gradient(ped, ep_x:float, ep_y:float, peds:list, ped_cost:float, obs:list, ob_cost:float, bxs:list, bx_rad:float, bx_cost:float, d:float, l:float):
    #sample costs around ped
    cxn = cost(ped.pos.x - d, ped.pos.z, ped, ep_x, ep_y, peds, ped_cost, obs, ob_cost, bxs, bx_rad, bx_cost, d)
    cxp = cost(ped.pos.x + d, ped.pos.z, ped, ep_x, ep_y, peds, ped_cost, obs, ob_cost, bxs, bx_rad, bx_cost, d)
    cyn = cost(ped.pos.x, ped.pos.z - d, ped, ep_x, ep_y, peds, ped_cost, obs, ob_cost, bxs, bx_rad, bx_cost, d)
    cyp = cost(ped.pos.x, ped.pos.z + d, ped, ep_x, ep_y, peds, ped_cost, obs, ob_cost, bxs, bx_rad, bx_cost, d)
    
    #find cost changes relative to x and y
    dx = (cxp - cxn) / (d * 2)
    dy = (cyp - cyn) / (d * 2)
    
    #get vector of change
    dv = vector(dx,0,dy)
    
    #normalize vector if mag greater than 1
    if dv.mag > 1:
        dv = dv.norm()
    
    #set x and y changes in case of normalization
    dx = dv.x
    dy = dv.z
    
    #return movements
    return (dx * l * -1), (dy * l * -1)

In [30]:
#create scene
scene = canvas()

#define space size
grid_x = 100
grid_y = 100

#define cost variables
ob_cost = 0.5
ped_cost = 0.05
bx_cost = 0.5
bx_rad = 0.5

#create plane
ground = box(pos=vector(0,0,0), size=vector(grid_x,1,grid_y))

#create goal
ep_x = -15
ep_y = 0

ep = cylinder(pos=vector(ep_x,0.5,ep_y),axis=vector(0,2,0),radius=0.05,color=vector(0,0,1))


#create pedestrains
peds = []
#peds.append(sphere(pos=vector(9,1,2),size=vector(1,1,1),radius=0.5))
#peds.append(sphere(pos=vector(9,1,-3.75),size=vector(1,1,1),radius=0.5))

for _ in range(50):
    ped_x = 9 + (r.random()) * 10
    ped_y = (r.random() - 0.5) * 10

    peds.append(sphere(pos=vector(ped_x,1,ped_y),size=vector(1,1,1),radius=0.5))

#create obstacles
obs = []
obs.append(cylinder(pos=vector(3,0.5,0),axis=vector(0,2,0),radius=0.5,color=vector(1,0,0)))
 
#create walls
bxs = []
bxs.append(box(pos=vector(-1,1.5,22.5),axis=vector(0,0,1),length=40,height=2,width=3,color=vector(1,0,0)))
bxs.append(box(pos=vector(-1,1.5,-22.5),axis=vector(0,0,1),length=40,height=2,width=3,color=vector(1,0,0)))


#wait for user to be ready then start
sleep(2)
waitforuser('i')
print('start')

#for 30 seconds at 24 fps, simulate
for i in range(24 * 30):
    sleep(1/24)
    
    #gets total changes in ped pos
    tdx = 0
    tdz = 0
    
    #for each ped get movement based on gradient
    for ped in peds:
        dx, dz = gradient(ped, ep.pos.x, ep.pos.z, peds, ped_cost, obs, ob_cost, bxs, bx_rad, bx_cost, 0.5, 0.5)
       
        tdx += dx
        tdz += dz
        
        ped.pos.x += dx
        ped.pos.z += dz
    
    #capture frame
    scene.capture(f'door_frame{i}.png')
    
    #if no major movement is done, end sim
    if abs(tdx) < 0.005 and abs(tdz) < 0.005:
        break
            
print('done')

<IPython.core.display.Javascript object>

start


KeyboardInterrupt: 

In [29]:
#create scene
scene = canvas()

#define space
grid_x = 100
grid_y = 100

#define cost variables
ob_cost = 0.5
ped_cost = 0.05
bx_cost = 5
bx_rad = 0.5

#create plane
ground = box(pos=vector(0,0,0), size=vector(grid_x,1,grid_y))

#create goal
ep_x = -20
ep_y = 0

ep = cylinder(pos=vector(ep_x,0.5,ep_y),axis=vector(0,2,0),radius=0.05,color=vector(0,0,1))


#create pedestrians
peds = []
#peds.append(sphere(pos=vector(9,1,2),size=vector(1,1,1),radius=0.5))
#peds.append(sphere(pos=vector(9,1,-3.75),size=vector(1,1,1),radius=0.5))

#group a
for _ in range(50):
    ped_x = 12 + (r.random()) * 10
    ped_y = (r.random() - 0.5) * 10

    peds.append(sphere(pos=vector(ped_x,1,ped_y),size=vector(1,1,1),radius=0.5))

#group b
for _ in range(50):
    ped_x = (r.random() - 0.25) * 10
    ped_y = -12 - (r.random()) * 10

    peds.append(sphere(pos=vector(ped_x,1,ped_y),size=vector(1,1,1),radius=0.5))

#create obstacles
obs = []

#create walls
bxs = []
bxs.append(box(pos=vector(0,1.5,22.5),axis=vector(0,0,1),length=40,height=2,width=20,color=vector(1,0,0)))
bxs.append(box(pos=vector(6.25,1.5,-5),axis=vector(0,0,1),length=5,height=2,width=7.5,color=vector(1,0,0)))
bxs.append(box(pos=vector(-6.25,1.5,-6.25),axis=vector(0,0,1),length=7.5,height=2,width=7.5,color=vector(1,0,0)))
bxs.append(box(pos=vector(-7.5,1.5,-25),axis=vector(0,0,1),length=30,height=2,width=5,color=vector(1,0,0)))
bxs.append(box(pos=vector(7.5,1.5,-23.75),axis=vector(0,0,1),length=32.5,height=2,width=5,color=vector(1,0,0)))


#wait for user to start
sleep(2)
waitforuser('i')
print('start')


#for 10 seconds at 24 fps, simulate
for i in range(24 * 10):
    sleep(1/24)
    
    #total movement of pedestrians
    tdx = 0
    tdz = 0
    
    #for each pedestrian get movement from gradient
    for ped in peds:
        dx, dz = gradient(ped, ep.pos.x, ep.pos.z, peds, ped_cost, obs, ob_cost, bxs, bx_rad, bx_cost, 0.5, 0.5)
       
        tdx += dx
        tdz += dz
        
        ped.pos.x += dx
        ped.pos.z += dz
    
    #capture frame
    scene.capture(f'merge_frame{i}.png')
    
    #if no major movement happens, end sim
    if abs(tdx) < 0.005 and abs(tdz) < 0.005:
        break
            
print('done')


<IPython.core.display.Javascript object>

start
done
